# Azure Language - Custom Question Answering

This notebook demonstrates how to use the Azure Language Question Answering API to
answer questions based on text records fetched from Tech901's website.

## Prerequisites

Set the following environment variables:
- `FOUNDRY_ENDPOINT` - Azure Language service endpoint
- `FOUNDRY_API_KEY` - API key for authentication

In [ ]:
import json
import os
from pathlib import Path

import requests
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

## Load Sample Data

Load Tech901 website content from the sample data files.

In [ ]:
data_dir = Path("03-QuestionAnswering-data")

# Load all text records
text_records = []
for i, filepath in enumerate(sorted(data_dir.glob("*.txt")), 1):
    with open(filepath, encoding="utf-8") as f:
        content = f.read()
    text_records.append({"id": str(i), "text": content})
    print(f"Loaded: {filepath.name} ({len(content)} chars)")

print(f"\nTotal records: {len(text_records)}")

## API Setup

Configure the Azure Language Question Answering API endpoint and authentication.

In [ ]:
endpoint = os.environ["FOUNDRY_ENDPOINT"]
api_key = os.environ["FOUNDRY_API_KEY"]

# Construct the Question Answering API URL
api_url = f"{endpoint}/language/:query-text?api-version=2023-04-01"

headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/json",
}

print(f"API Endpoint: {endpoint}")
print("API Key: [CONFIGURED]")

## Define Question

Enter a question to ask about Tech901's programs.

In [ ]:
# Example questions you can try:
# - "How much does the IT Foundations course cost?"
# - "What are Tech901's office hours?"
# - "How long is the IT Networking course?"
# - "Can I get a refund if I withdraw from a class?"
# - "What certifications does Tech901 prepare students for?"

question = "How much does the IT Foundations course cost?"
print(f"Question: {question}")

## Make API Call

Send the question and text records to the Question Answering API.

In [ ]:
# Build the request payload
payload = {"question": question, "records": text_records, "language": "en"}

# Make the API request
response = requests.post(api_url, headers=headers, json=payload, timeout=30)
response.raise_for_status()
result = response.json()

print("API call successful!")

## Raw API Results

Display the complete JSON response from the API.

In [ ]:
print(json.dumps(result, indent=2))

## Human-Friendly Results

Present the question answering results in a readable format.

In [ ]:
console = Console()

# Display the question
console.print(Panel(f"[bold cyan]{question}[/bold cyan]", title="Question"))

answers = result.get("answers", [])

if not answers:
    console.print("[yellow]No answers found for this question.[/yellow]")
else:
    # Create answers table
    table = Table(title="Answers Found")
    table.add_column("Rank", style="cyan", justify="center", width=6)
    table.add_column("Confidence", justify="center", width=12)
    table.add_column("Answer", style="green", max_width=80)

    for i, answer in enumerate(answers[:5], 1):
        confidence = f"{answer['confidenceScore']:.0%}"
        answer_text = answer.get("answer", "N/A")

        # Truncate long answers for display
        if len(answer_text) > 200:
            answer_text = answer_text[:200] + "..."

        table.add_row(str(i), confidence, answer_text)

    console.print(table)

    # Show the best answer with more detail
    best_answer = answers[0]
    answer_span = best_answer.get("answerSpan")

    detail = f"""
[bold]Best Answer:[/bold]
{best_answer.get("answer", "N/A")}

[bold]Confidence Score:[/bold] {best_answer["confidenceScore"]:.1%}
[bold]Source Record ID:[/bold] {best_answer.get("id", "N/A")}
"""

    if answer_span and answer_span.get("text"):
        detail += f"\n[bold]Key Span:[/bold] [cyan]{answer_span['text']}[/cyan]"

    console.print(Panel(detail, title="Top Answer Details"))

## Try More Questions

Function to easily ask additional questions.

In [ ]:
def ask_question(q: str) -> None:
    """Ask a question and display the answer."""
    payload = {"question": q, "records": text_records, "language": "en"}
    resp = requests.post(api_url, headers=headers, json=payload, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    console.print(f"\n[bold cyan]Q: {q}[/bold cyan]")

    if data.get("answers"):
        best = data["answers"][0]
        console.print(f"[green]A: {best['answer'][:300]}[/green]")
        console.print(f"[dim]Confidence: {best['confidenceScore']:.0%}[/dim]")
    else:
        console.print("[yellow]No answer found.[/yellow]")


# Try additional questions
ask_question("What are Tech901's office hours?")
ask_question("How long is the IT Security course?")
ask_question("What is the refund policy?")